In [ ]:
import pandas as pd
from IPython.core.display import display

In [ ]:
pkl_src_path = '../data/dataframes/eng_suc_sim' #'../data/dataframes/coref_with_translations_1000.pkl'
conll_file_dest_path = '../allennlp-model/data/test/test_sim.english.conll' #'../allennlp-model/data/train/train.english_1000.conll'

In [ ]:
df = pd.read_pickle(pkl_src_path)
display(df)

In [ ]:
def insert_or_append(d, key, value):
    if key in d:
        d[key] = d.get(key) + '|' + value
    else:
        d[key] = value
    return d

coref_clusters = df['Coref_Clusters']
conll_coref_dicts = []

for clusters in coref_clusters:
    conll_cluster_count = 0
    conll_coref_dict = {}
    for cluster in clusters:
        for [span_start, span_end] in cluster:
            conll_value = ''
            if span_start == span_end:
                conll_value = '(' + str(conll_cluster_count) + ')'
                insert_or_append(conll_coref_dict, span_start, conll_value)
            else:
                conll_start_value = '(' + str(conll_cluster_count)
                conll_end_value = str(conll_cluster_count) + ')'
                insert_or_append(conll_coref_dict, span_start, conll_start_value)
                insert_or_append(conll_coref_dict, span_end, conll_end_value)
        conll_cluster_count += 1
    conll_coref_dicts.append(conll_coref_dict)


In [ ]:
path = conll_file_dest_path
texts = df['Text']
full_ids = df['Text_ID']
print(len(conll_coref_dicts))
with open(path, 'w') as file:
    for text, full_id, corefs in zip(texts, full_ids, conll_coref_dicts):
        try:
            [doc_id, text_id] = full_id.split('-')
            text_idx = 0
            file.write('#begin document (%s); part %s\n' %(doc_id, text_id.zfill(3)))
            for sentence in text:
               padding = len(max(sentence, key=len))
               for idx, word in enumerate(sentence):
                   coref = corefs.get(text_idx, '-')
                   file.write(doc_id + '\t')
                   file.write(text_id + '\t')
                   file.write(str(idx) + '\t')
                   file.write('{:{padding}s}'.format(word, padding=padding+1))
                   file.write('XX\t')
                   file.write('-' + '\t' + '-' + '\t' + '-' + '\t' + '-' + '\t' + '-' + '\t')
                   # file.write('{:40s}'.format(df.loc[i, 'Speaker']))
                   file.write('*' + '\t' + '*' + '\t')
                   file.write(coref)
                   file.write('\n')
                   text_idx += 1
               file.write('\n')
            file.write('\n#end document\n')
        except:
            print(text)